# Определение языка и VK API

В данном задании вам нужно будет:

* используя API Вконтакте, скачать комментарии к первым ста постам из пяти сообществ
* натренировать модель распознавания языков на статьях из Википедии.
* распознать язык всех комментариев, где в тексте есть 10 и более символов, и построить статистику

## VK API

Для подключения к ВКонтакте мы будем использовать VK API. Здесь есть документация к этой библиотеке https://vk-api.readthedocs.io/en/latest/

In [1]:
import wikipedia
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import json
import re
import requests


In [4]:
TOKEN = ''

params = {'access_token': TOKEN,
          'v': '5.95'}

### К сожалению, документация предоженной вами библиотеки не дала мне уверенности в том, что мой пароль не попадет туда, куда я не хочу. Поэтому мой код работает на токене vk.

In [5]:
def vk(method, params):
    api_req = 'https://api.vk.com/method/'
    api_req += method + '?'
    api_req += '&'.join(['{}={}'.format(key, params[key]) for key in params])
    json_raw = requests.get(api_req).text
    return json.loads(json_raw)

In [6]:
# список domain'ов, чтобы вам не копировать их самими :) спасибо :***
publics = ["futureisnow",
           "eternalclassic",
           "ukrlit_memes",
           "ukrainer_net",
           "amanzohel",
           "barg_kurumk_culture"]

g_dmns = ','.join(publics)
gidpars = params.copy()
gidpars['group_ids'] = g_dmns

In [7]:
rawids = vk('groups.getById', gidpars)
g_ids = {}
for i in rawids['response']:
    gid = -1 * i['id']
    name = i['name']
    g_ids[gid] = name
    
print(g_ids)

{-111587102: 'киберпанк, который мы заслужили', -129440544: 'eternal classic', -131348832: 'Файні меми про українську лiтературу', -5164516: 'Ukraїner', -66347916: 'Буряад аман зохёол', -95189895: 'Баргажанай буряадууд (Хурамхаан, Баргажан)'}


In [8]:
def wall_get(params):
    m = 'wall.get'
    posts = {}
    result = vk(m, params)
    items = result['response']['items']
    p_ids = []
    for p in items:
        p_ids.append(p["id"])
    return p_ids

In [9]:
pre_corpora = {}

params['count'] = '100'

for dom in g_ids:
    params['owner_id'] = str(dom)
    posts = wall_get(params)
    pre_corpora[dom] = posts

In [10]:
def comments_get(params, posts):
    m = 'wall.getComments'
    comments = []
    for post_id in posts:
        params['post_id'] = post_id
        result = vk(m, params)
        items = result['response']['items']
        for item in items:
            if 'text' in item.keys():
                text = item['text']
                if len(text) > 10:
                    comments.append(text)
    return comments

In [11]:
corpora = {}

for dom in pre_corpora:
    params['owner_id'] = str(dom)
    p_ids = pre_corpora[dom]
    posts = comments_get(params, p_ids)
    corpora[g_ids[dom]] = posts
    
for c in corpora:
    print('%d\t%s' % (len(corpora[c]), c))

1180	киберпанк, который мы заслужили
728	eternal classic
249	Файні меми про українську лiтературу
199	Ukraїner
255	Буряад аман зохёол
232	Баргажанай буряадууд (Хурамхаан, Баргажан)


## Тренировка моделей

В наших комментариях встречались русский, украинский, английский и бурятский.

In [12]:
test_langs = ('ru', 'uk', 'en', 'bxr')

Скачайте документы, на которых вы будете обучать свои модели. Для наших целей хорошо иметь для каждого языка корпус размером около 50 статей.

In [13]:
def get_texts_for_lang(lang, n=40):
    wiki_content = []
    wikipedia.set_lang(lang)
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
            wiki_content.append("%s\n%s" % (page.title, page.content.replace('=', '')))
        except wikipedia.exceptions.WikipediaException:
            pass
    return wiki_content

In [14]:
wiki_texts = {}

for lang in test_langs:
    wcon = get_texts_for_lang(lang)
    wiki_texts[lang] = wcon

C:\Users\admin\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\admin\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [15]:
from collections import Counter

Сделайте определялку на частотах слов.

In [16]:
def collect_freqlist(wiki_pages, max_len=100):
    freqlist = Counter()
    # не забудем про токенизацию - nltk.word_tokenize
    for text in wiki_pages:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha():
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

In [17]:
freqs = {}


for lang in test_langs:
    freqs[lang] = collect_freqlist(wiki_texts[lang])

In [18]:
def simple_lang_detect(freq_lists, text):
    counts = Counter()
    for lang, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[lang] += int(freq_list[word] > 0)
    return counts.most_common()

Сделайте определялку на символьных энграммах.

In [ ]:
!pip install matplotlib==3.0.3
!pip install seaborn

In [20]:
from sklearn import feature_extraction

In [21]:
from sklearn import pipeline
from sklearn import naive_bayes
import numpy as np

%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [22]:
clf = pipeline.Pipeline([
    ('vctr', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2), analyzer='char')),
    ('clf', naive_bayes.MultinomialNB())
])

In [23]:
all_texts = []
lang_indices = []
for lang in wiki_texts:
    all_texts.extend(wiki_texts[lang])
    lang_indices.extend([lang]*len(wiki_texts[lang]))

In [24]:
# Обучаем классификатор
clf.fit(np.array(all_texts), np.array(lang_indices))

Pipeline(memory=None,
     steps=[('vctr', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [25]:
clf.predict(all_texts)


array(['ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'en', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'ru', 'ru', 'ru', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'ru', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr',
       'bxr', 'bxr', 'ru', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr',
       'bxr', 'bxr', 'bxr', 'ru', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr',
       'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', '

# Определение языка

Определите язык каждого комментария в каждом паблике с помощью определялки на частотах слов и покажите доли языков среди комментариев для каждого паблика.

In [19]:
lang_detects_freqs = {}

for pub in corpora:
    lang_detects_freqs[pub] = {}
    for text in corpora[pub]:
        options = simple_lang_detect(freqs, text)
        if len(options) > 0:
            l = options[0][0]
            if l in lang_detects_freqs[pub]:
                lang_detects_freqs[pub][l] += 1
            else:
                lang_detects_freqs[pub][l] = 1

for pub in lang_detects_freqs:
    print('---\t' + pub + '\n')
    p_info = lang_detects_freqs[pub]
    total = 0
    for l in p_info:
        total += p_info[l]        
    for l in test_langs:
        if l in p_info:
            k = round(p_info[l] / total, 2)
            q = p_info[l]
        else:
            k = 0.00
            q = 0
        print('%s\t%d\t%s' % (l, q, str(k)))
    print('total:\t%d\t%s\n' % (total, '1.00'))

---	киберпанк, который мы заслужили

ru	1161	0.98
uk	7	0.01
en	12	0.01
bxr	0	0.0
total:	1180	1.00

---	eternal classic

ru	293	0.4
uk	0	0.0
en	429	0.59
bxr	5	0.01
total:	727	1.00

---	Файні меми про українську лiтературу

ru	168	0.67
uk	77	0.31
en	4	0.02
bxr	0	0.0
total:	249	1.00

---	Ukraїner

ru	122	0.61
uk	76	0.38
en	0	0.0
bxr	1	0.01
total:	199	1.00

---	Буряад аман зохёол

ru	245	0.96
uk	1	0.0
en	0	0.0
bxr	9	0.04
total:	255	1.00

---	Баргажанай буряадууд (Хурамхаан, Баргажан)

ru	210	0.91
uk	1	0.0
en	0	0.0
bxr	21	0.09
total:	232	1.00



Сделайте то же самое для определителя на символьных энграммах.

In [37]:
for pub in corpora:
    lang_ratio = Counter(clf.predict(corpora[pub]))
    
    print('---\t' + pub + '\n')
    total = 0
    for lang in lang_ratio:
        total += lang_ratio[lang]        
    for lang in test_langs:
        if lang in lang_ratio:
            k = round(lang_ratio[lang] / total, 2)
            q = lang_ratio[lang]
        else:
            k = 0.00
            q = 0
        print('%s\t%d\t%s' % (lang, q, str(k)))
    print('total:\t%d\t%s\n' % (total, '1.00'))

---	киберпанк, который мы заслужили

ru	1077	0.91
uk	0	0.0
en	44	0.04
bxr	59	0.05
total:	1180	1.00

---	eternal classic

ru	0	0.0
uk	0	0.0
en	726	1.0
bxr	2	0.0
total:	728	1.00

---	Файні меми про українську лiтературу

ru	131	0.53
uk	97	0.39
en	6	0.02
bxr	15	0.06
total:	249	1.00

---	Ukraїner

ru	85	0.43
uk	94	0.47
en	4	0.02
bxr	16	0.08
total:	199	1.00

---	Буряад аман зохёол

ru	36	0.14
uk	0	0.0
en	0	0.0
bxr	219	0.86
total:	255	1.00

---	Баргажанай буряадууд (Хурамхаан, Баргажан)

ru	122	0.53
uk	0	0.0
en	1	0.0
bxr	109	0.47
total:	232	1.00



Обсудите работу каждого из классификаторов, обсудите ошибки, объясните разницу в результатах.

Определитель по частотным словам может ориентироваться на числа, слова из пары символов или смайлики. Поэтому энграммный определитель выглядит точнее.